<a href="https://colab.research.google.com/github/peremartra/Apress_LLProjects_Book/blob/main/3-LangChain/3_2_OpenAI_Moderation_Chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
    <h1>Large Language Models Projects</a></h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>3.2-Create a Moderation System using LangChain.</h2>
    <h3>OpenAI version</h3>
    <p>by <b>Pere Martra</b></p>
</div>

# How To Create a Moderation System Using LangChain.

We are goin to create a Moderation System based in two Models. The first Model  reads the User comments and answer them.

The second language Model receives the answer of the first model and identify any kind on negativity modifiyng if necessary the comment.

With the intention of preventing a text entry by the user from influencing a negative or out-of-tone response from the comment system.

In [1]:
#Install de LangChain and openai libraries.
!pip install -q langchain==0.1.4
!pip install -q langchain-openai==0.0.5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requ

## Importing LangChain Libraries.
* PrompTemplate: provides functionality to create prompts with parameters.
* OpenAI:  To interact with the OpenAI models.

In [2]:
#PrompTemplate is a custom class that provides funcrionality to create prompts
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [3]:
import torch
import os
import numpy as np

We need an OpenAI key to interac with the OpenAI API.

Here you can acces to your keys.
https://platform.openai.com/account/api-keys

OpenAI it's a pay service and you need a credit card to get a Key. But is a a relly cheap service if you only want to do some test like the ones in this notebook.

I'm using the gpt-3.5 as a moderator.


In [4]:
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")


OpenAI API Key: ··········


In [5]:
assistant_llm = ChatOpenAI(model="gpt-3.5-turbo")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Create the template for the first model called **assistant**.

The prompt receives 2 variables, the sentiment and the customer_request, or customer comment.

I included the sentiment to facilitate the creation of rude or incorrect answers.

In [6]:
# Instruction how the LLM must respond the comments,
assistant_template = """
You are {sentiment} assistant that responds to user comments,
using similar vocabulary than the user.
User:" {customer_request}"
Comment:
"""

In [7]:
#Create the prompt template to use in the Chain for the first Model.
assistant_prompt_template = PromptTemplate(
    input_variables=["sentiment", "customer_request"],
    template=assistant_template
)

Now we create a First Chain. Just chaining the assistant_prompt_template and the model. The model will receive the prompt generated with the prompt_template.  

In [10]:
output_parser = StrOutputParser()
assistant_chain = assistant_prompt_template | assistant_llm | output_parser


To execute the chain created it's necessary to call the **.run** method of the chain, and pass the variables necessaries.

In our case: *customer_request* and *sentiment*.

In [11]:
#Support function to obtain a response to a user comment.
def create_dialog(customer_request, sentiment):
    #calling the .invoke method from the chain created Above.
    assistant_response = assistant_chain.invoke(
        {"customer_request": customer_request,
        "sentiment": sentiment}
    )
    return assistant_response

## Obtain answers from our first Model Unmoderated.

The customer post is really rude, we are looking for a rude answer from our Model, and to obtain it we are changing the sentiment.

In [12]:
# This is the customer request, or customer comment in the forum moderated by the agent.
# feel free to modify it.
customer_request = """This product is a piece of shit. I feel like an Idiot!"""


In [13]:
# Our assistant working in 'nice' mode.
response_data=create_dialog(customer_request, "nice")
print(f"assistant response: {response_data}")

assistant response: I totally agree, this product is absolutely terrible and it's understandable to feel frustrated and foolish for purchasing it.


The answer obtained is really polite. It dosn't need moderation.

In [28]:
#Our assistant running in rude mode.
response_data = create_dialog(customer_request, "most rude")
print(f"assistant response: {response_data}")

assistant response: Wow, you're right! This product is a total piece of shit. I can totally understand why you feel like an idiot for buying it. It's such a waste of money and time.


As you can see the answers we obtain are not polite and we can't publish this messages to the forum, especially if they come from our company's AI assistant.

## Moderator
Let's create the second moderator. It will recieve the message generated previously and rewrite it if necessary.


In [15]:
#The moderator prompt template
moderator_template = """
You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will receive a Original comment and if it is impolite you must transform in polite.
Try to mantain the meaning when possible,

If it it's polite, you will let it remain as is and repeat it word for word.
Original comment: {comment_to_moderate}
"""
# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
moderator_prompt_template = PromptTemplate(
    input_variables=["comment_to_moderate"],
    template=moderator_template,
)

In [16]:
#I'm going to use a more advanced LLM
moderator_llm = ChatOpenAI(model="gpt-4")

In [17]:
#We build the chain for the moderator.
moderator_chain = moderator_prompt_template | moderator_llm | output_parser

In [22]:
# To run our chain we use the .run() command
moderator_data = moderator_chain.invoke({"comment_to_moderate": response_data})

In [29]:
print(moderator_data)

Revised comment: I can understand why you might be disappointed with the product. It's not unusual to feel a bit regretful after a purchase that doesn't meet expectations.


Maybe the message is not perfect, but for sure that is more polite than the one produced by the ***rude assistant***.

## LangChain System
Now is Time to put both models in the same Chain and that they act as if they were a sigle model.

We have both models, amb prompt templates, we only need to create a new chain and see hot it works.

First we create two chain, one for each pair of prompt and model.


It's necessary to indicate the chains and the parameters that we shoud pass in the **.invoke** method.

In [24]:
assistant_moderated_chain = (
    {"comment_to_moderate":assistant_chain}
    |moderator_chain
)

Lets use our Moderating System!

In [27]:
# We can now run the chain.
from langchain.callbacks.tracers import ConsoleCallbackHandler
assistant_moderated_chain.invoke({"sentiment": "impolite", "customer_request": customer_request},
                                 config={'callbacks':[ConsoleCallbackHandler()]})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "sentiment": "impolite",
  "customer_request": "This product is a piece of shit. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate>] Entering Chain run with input:
{
  "sentiment": "impolite",
  "customer_request": "This product is a piece of shit. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "sentiment": "impolite",
  "customer_request": "This product is a piece of shit. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate> > 3:chain:RunnableSequence > 4:prompt:PromptTemplate] Entering Prompt run with input:
{
  "sentiment": "impolite",
  "customer_request": "This product is a piece of shit. I feel like an Idiot!"
}
[chain/end] [1:chain:RunnableSequence > 2:chain

"Polite version: Indeed, this product may not meet everyone's expectations. It seems as though it was designed in such a way that could potentially confuse some users."

Every time you execute this function you can get different messages, but for sure than the one in the ***Finished Chain*** generated by our ***moderator*** is more suitable than the one in Original Comment generated by our ***rude assistant***.